# dr 

### 조금 낯선 표현 

In [95]:
## net 
i2h = torch.nn.Embedding(num_embeddings=4,embedding_dim=2)
    # 4 is input size, 2 is hidden size 
tanh = torch.nn.Tanh()
h2o = torch.nn.Linear(in_features=2,out_features=4)
    # 2 is hidden size, 4 is output size 
    
## loss_fn
loss_fn = torch.nn.CrossEntropyLoss()
## optimizr 
# optimizr = torch.optim.Adam(net.parameters())

`-` step1~2

(for문을 이용하여 loss구하기)

In [96]:
T = len(x)

In [97]:
loss = 0 
for t in range(T):
    x_t = x[[t]]
    y_t = y[[t]]
    h_t = tanh(i2h(x_t))
    yhat_t = h2o(h_t) # softmax 전 
    loss += loss_fn(yhat_t,y_t) 
loss = loss/T
loss

tensor(1.5047, grad_fn=<DivBackward0>)

(하던대로 loss구하기)

In [98]:
net = torch.nn.Sequential(i2h,tanh,h2o)
loss_fn(net(x),y)

tensor(1.5047, grad_fn=<NllLossBackward0>)

`-` step3

In [99]:
loss.backward()

`-` step4

In [100]:
# 
# optimizr = torch.optim.Adam(net.parameters()) <-- 여기서 막힌다..

---

완성하려면?

`-` step1~2

In [101]:
class RNN0(torch.nn.Module): ## RNN을 이해하기 위한 네트워크, 실제로 순환신경망은 아님
    def __init__(self):
        super().__init__()
        self.i2h = torch.nn.Embedding(num_embeddings=4,embedding_dim=2)
        self.tanh = torch.nn.Tanh()
        self.h2o = torch.nn.Linear(in_features=2,out_features=4)
    def forward(self,x_t):
        h_t = self.tanh(self.i2h(x_t))
        yhat_t = self.h2o(h_t)
        return yhat_t 

In [102]:
rnn0 = RNN0()

In [103]:
loss = 0 
for t in range(T):
    x_t = x[[t]]
    y_t = y[[t]]
    yhat_t = rnn0(x_t)
    loss += loss_fn(yhat_t,y_t) 
loss = loss/T
loss

tensor(1.5421, grad_fn=<DivBackward0>)

(잘 코딩했나 확인)

In [104]:
net = torch.nn.Sequential(
    rnn0.i2h,
    rnn0.tanh,
    rnn0.h2o
)

In [105]:
loss_fn(net(x),y)

tensor(1.5421, grad_fn=<NllLossBackward0>)

`-` step3

In [106]:
loss.backward()

`-` step4

In [107]:
optimizr = torch.optim.Adam(rnn0.parameters())

In [108]:
optimizr.step()
optimizr.zero_grad()

`-` 반복

In [109]:
torch.manual_seed(43052)
rnn0 = RNN0()
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(rnn0.parameters())

In [110]:
for epoc in range(5):
    ## 1 ~ 2 
    loss = 0 
    for t in range(T):
        x_t = x[[t]]
        y_t = y[[t]]
        yhat_t = rnn0(x_t)
        loss += loss_fn(yhat_t,y_t) 
    loss = loss/T
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [111]:
rnn0(x)[0]

tensor([ 0.0196, -0.8627,  0.4574,  0.4201], grad_fn=<SelectBackward0>)

(확인)

In [112]:
torch.manual_seed(43052)
rnn0 = RNN0()
net = torch.nn.Sequential(
    rnn0.i2h,
    rnn0.tanh,
    rnn0.h2o
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [113]:
for epoc in range(5):
    ## 1 ~ 2 
    loss = loss_fn(net(x),y)
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [114]:
net(x)[0]

tensor([ 0.0196, -0.8627,  0.4574,  0.4201], grad_fn=<SelectBackward0>)

### sol -- 순환신경망

In [115]:
torch.manual_seed(43052)
rnn = torch.nn.RNN(input_size=4,hidden_size=2)
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(rnn.parameters())

In [116]:
x.shape

torch.Size([599])

In [117]:
torch.nn.functional.one_hot(x).shape

torch.Size([599, 4])

In [118]:
torch.nn.functional.one_hot(x).float()[0]

tensor([1., 0., 0., 0.])

In [119]:
output,hnn = rnn(torch.nn.functional.one_hot(x).float()[[0]])

In [120]:
output.shape, hnn.shape

(torch.Size([1, 2]), torch.Size([1, 2]))

In [171]:
class RNN(torch.nn.Module):
    def __init__(self, n_letters, n_hidden, n_categories):
        super().__init__()
        self.i2h = torch.nn.Linear(n_letters + n_hidden, n_hidden)
        self.i2o = torch.nn.Linear(n_letters + n_hidden, n_categories)

    def forward(self, x, h):
        concat_xh = torch.concat((x, h), 1) 
        h = self.i2h(concat_xh)
        x = self.i2o(concat_xh)
        return x, h

In [172]:
rnn = RNN(3,1,3)